<a href="https://colab.research.google.com/github/bh-Range/image_data_all/blob/main/%E7%AB%9E%E5%93%81%E5%88%86%E6%9E%90_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
!pip install pytorch

In [ ]:
!pip install torchvision

In [4]:
!pip install sklearn

In [ ]:
import numpy as np

In [ ]:
!pip install pretrainedmodels

In [ ]:
!pip install keras

In [64]:
!git clone https://github.com/bh-Range/image_data_all.git

Cloning into 'image_data_all'...
remote: Enumerating objects: 11543, done.
remote: Counting objects: 100% (11543/11543), done.
remote: Compressing objects: 100% (9546/9546), done.
remote: Total 11543 (delta 16), reused 11540 (delta 16), pack-reused 0
Receiving objects: 100% (11543/11543), 163.25 MiB | 33.89 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Checking out files: 100% (7819/7819), done.


In [63]:
import numpy as np
from numpy import linalg as LA
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input


class VGGNet:
  def __init__(self):
    self.input_shape = (224, 224, 3)
      self.model = VGG16(weights='imagenet', pooling='max', include_top=False,
                           input_shape=(self.input_shape[0], self.input_shape[1], self.input_shape[2]))

  def extract_feat(self, img_path):
    '''提取图像特征

      :param img_path: 图像路径
      :return: 归一化后的图像特征
    '''
    img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    feat = self.model.predict(img)
    norm_feat = feat[0] / LA.norm(feat[0])
    return norm_feat

0.63770986


In [77]:
import os
def traverse_dir(current_dir):
    '''
    current_dir: 绝对路径或当前路径，例如 'D:/picture'
    '''
    filedir_list=[]
    file_list = os.listdir(current_dir)
    for file in file_list:
        path = os.path.join(current_dir, file)
        '''
        if os.path.isfile(path):
            filedir_list.append(path)
        '''
        filedir_list.append(path)
    return filedir_list          

In [90]:
#os.listdir(filedir_list[1])
def image_name(filedir_list):
  '''
  输出为字典，键值对为：产品名：图片路径
  filedir_list:输入的列表，存储某一小类下所有产品的文件夹名
  '''
  dict_image=dict()
  for file in filedir_list:
    #图片名
    name=file.split("/")[-1]

    #遍历文件夹下的文件
    file_list=os.listdir(file)

    if len(file_list)>0:
      path=os.path.join(file, file_list[0])
      dict_image[name]=path

  return dict_image

In [96]:
#字典：产品名：图片路径
dict_image=image_name(filedir_list)

#列表：产品名
product_name=[key for key in dict_image]

In [100]:
def image_sim(name1,name2,dict_image,model):
  '''
  计算两个产品之间的相似性
  name1/name2:产品名
  model:模型
  '''
  #model = VGGNet()
  path1,path2=dict_image[name1],dict_image[name2]
  image1 = model.extract_feat(path1)
  image2 = model.extract_feat(path2)
  return np.dot(image1, image2.T)

In [116]:
def topk_image_sim(name,dict_image,model,k=5):
  '''
  计算与某个产品最相似的前k个产品，返回（产品名，相似度），输出为列表
  '''
  res=[]
  count=0
  for key in dict_image:
    try:
      sim=image_sim(name,key,dict_image,model)
      res.append([key,sim])
    except:
      print("{} image cannot open".format(key))
  res=sorted(res,key=lambda x:x[1],reverse=True)
  return res

In [124]:
if __name__ == '__main__':
  #大类、小类
  big_class="Skirt"
  tiny_class="Skirt"

  #对应文件路径
  current_dir="/content/image_data_all/picture_VERO_MODA/"+big_class+"/"+tiny_class

  filedir_list=traverse_dir(current_dir)

  #字典：产品名：图片路径
  dict_image=image_name(filedir_list)

  #列表：产品名
  product_name=[key for key in dict_image]


  name="ACCESS MINI SKIRT(NR)"
  model=VGGNet()
  
  res_skirt=topk_image_sim(name,dict_image,model,k=5)


In [121]:
res_top_jersey[:5]

[['ADRIEN FEEL BOX SS TOP(UC)', 1.0000001],
 ['CLEO SS TOP(SS)', 0.8299022],
 ['FLOWER SS TOP(NN)', 0.8124581],
 ['PEARL RABBY SS TOP CL5(SL)', 0.80588883],
 ['VIBE SUN BOX SS JERSEY TOP(GS)', 0.80328166]]

In [125]:
res_skirt[:5]

[['ACCESS MINI SKIRT(NR)', 1.0],
 ['MOON MINI SKIRT(SL)', 0.82429695],
 ['TESS HW SKIRT(TT)', 0.8113109],
 ['RIGHT HW SKIRT(COUTUREMADE)', 0.8065959],
 ['TRACY MW MINI SKIRT(TT)', 0.79641855]]